In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
log_dir = '/Users/rosenasrawi/Documents/VU PhD/Projects/rn6 - Temporal action/Data/lab/logfiles/'
subjects = list(range(1,9)) + list(range(10, 27))

In [ ]:
def load_data(log_dir, subjects):

    dfs = []

    for s in subjects:

        # Load session files
        ses1 = 'log_s' + str(s) + 'a.csv'
        ses2 = 'log_s' + str(s) + 'b.csv'

        log1 = pd.read_csv(log_dir + ses1)
        log1.insert(0, 'session', 'a')

        log2 = pd.read_csv(log_dir + ses2)
        log2.insert(0, 'session', 'b')

        # Concatenate sessions
        log = pd.concat([log1, log2], axis = 0, ignore_index=True)
        log.insert(0, 'subject', s)

        # Find bad DT
        thres_DT1 = log['DT1'].mean() + 2.5 * log['DT1'].std()
        thres_DT2 = log['DT2'].mean() + 2.5 * log['DT2'].std()

        log['bad_DT'] = (log['DT1'] >= thres_DT1) | (log['DT2'] >= thres_DT2)
        
        dfs.append(log)

    log = pd.concat(dfs, axis = 0, ignore_index=True)

    return log

In [ ]:
# Get all subject data

log = load_data(log_dir, subjects)

print(log['bad_DT'].sum() / len(log['bad_DT']), 'proportion of trials marked as bad')

In [ ]:
# Get summaries

DT_all = pd.pivot_table(log, values = ['DT1', 'DT2'], index = 'subject',
                          columns = 'order', aggfunc = 'mean')

DT_mean = DT_all.mean(axis=0)
DT_se = DT_all.sem(axis=0)

err_all = pd.pivot_table(log, values = ['diff1', 'diff2'], index = 'subject',
                           columns = 'order', aggfunc = 'mean')

err_mean = err_all.mean(axis=0)
err_se = err_all.sem(axis=0)

In [ ]:
# Plot settings

mpl.rcParams['font.family'] = 'Helvetica Neue'

barWidth = 0.06
x = [0, 0.1, 0.25, 0.35]
xt, mid = [0.05, 0.3], 0.175

palette ={
    'blue': '#96bfe6',
    'green': '#8cdea1',
    'orange': '#ff7340',
    'yellow': '#facf73',
    'grey': '#999999'
}


In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

# data
ax.bar(x, err_mean, yerr = err_se, 
       color = [palette['blue'],palette['orange'],palette['blue'],palette['orange']], edgecolor = 'black', 
       width = [0.06]*4, linewidth = 1,
       error_kw = {'linewidth':1, 'capsize':3})

for i, ix in enumerate(x):
    ax.scatter([ix]*len(err_all), err_all.iloc[:, i], color = palette['grey'], s = 10)

for i1,i2 in [[0,1],[2,3]]:
    ax.plot([x[i1], x[i2]], [err_all.iloc[:,i1], err_all.iloc[:,i2]], 
            color = palette['grey'], linestyle = '-', linewidth = 0.5)

ax.axvline(x = mid, color = palette['grey'], linestyle = 'dashed', linewidth = 1)

# x and y
ax.set_xticks(xt, labels=['First response', 'Second response'], fontsize = 13)
ax.tick_params(axis = 'y', labelsize = 12)
ax.set_ylabel('Absolute error (deg)', fontweight = 'bold', fontsize = 14)
ax.set_ylim(0, 25)

# legend
ax.text(-0.03, 24, 'Same order', color = palette['blue'], fontsize = 12)
ax.text(-0.03, 22.5, 'Reversed order', color = palette['orange'], fontsize = 12)

# outlines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

# data
ax.bar(x, DT_mean, yerr = DT_se, 
       color = [palette['blue'],palette['orange'],palette['blue'],palette['orange']], edgecolor = 'black', 
       width = [0.06]*4, linewidth = 1,
       error_kw = {'linewidth':1, 'capsize':3})

for i, ix in enumerate(x):
    ax.scatter([ix]*len(DT_all), DT_all.iloc[:, i], color = palette['grey'], s = 10)

for i1,i2 in [[0,1],[2,3]]:
    ax.plot([x[i1], x[i2]], [DT_all.iloc[:,i1], DT_all.iloc[:,i2]], 
            color = palette['grey'], linestyle = '-', linewidth = 0.5)

ax.axvline(x = mid, color = palette['grey'], linestyle = 'dashed', linewidth = 1)

# x and y
ax.set_xticks(xt, labels=['First response', 'Second response'], fontsize = 13)
ax.tick_params(axis = 'y', labelsize = 12)
ax.set_ylabel('Decision time (ms)', fontweight = 'bold', fontsize = 14)
ax.set_yticks([0,0.25,0.5,0.75,1], [0,250,500,750,1000])
ax.set_ylim(0, 1)

# legend
ax.text(0.25, 0.9, 'Same order', color = palette['blue'], fontsize = 12)
ax.text(0.25, 0.83, 'Reversed order',  color = palette['orange'], fontsize = 12)

# outlines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
# Main effects

DT_main = pd.DataFrame(columns=['order','moment'])
DT_main.order = DT_all.loc[:, (slice(None), 'second')].mean(axis=1) - DT_all.loc[:, (slice(None), 'first')].mean(axis=1)
DT_main.moment = DT_all.DT1.mean(axis=1) - DT_all.DT2.mean(axis=1)

DT_main = pd.melt(DT_main, var_name='effect', value_name='DT')

err_main = pd.DataFrame(columns=['order','moment'])
err_main.order = err_all.loc[:, (slice(None), 'second')].mean(axis=1) - err_all.loc[:, (slice(None), 'first')].mean(axis=1)
err_main.moment = err_all.diff2.mean(axis=1) - err_all.diff1.mean(axis=1)

err_main = pd.melt(err_main, var_name='effect', value_name='diff')

In [ ]:
fig, ax = plt.subplots(figsize=(2, 5))

sns.violinplot(x='DT', y='effect', data=DT_main, ax=ax, inner = None,
               hue='effect', width=0.5, palette={'order': palette['yellow'], 'moment': palette['green']},
               edgecolor = 'black', linewidth= 1)

sns.swarmplot(x='DT', y='effect', data=DT_main,
              ax=ax, s=3, color = palette['grey'])

ax.axvline(x=0, color=palette['grey'], linestyle='dashed', linewidth=1)

ax.tick_params(axis='x', labelsize=12)
ax.set_xticks([0,0.25,0.5], [0,250,500])
ax.set_xlabel('Difference (ms)', fontweight='bold', fontsize=14)

ax.set_ylabel('')
ax.set_yticklabels(['Order', 'Moment'], fontsize=14,rotation=90,va='center')

ax.text(0.1, -0.1, 'Same vs reversed', color = palette['yellow'], fontsize = 12)
ax.text(0.3, 0.9, 'First vs second', color=palette['green'], fontsize = 12)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2, 5))

sns.violinplot(x='diff', y='effect', data=err_main, ax=ax, inner = None,
               hue='effect', width=0.5, palette={'order': palette['yellow'], 'moment': palette['green']},
               edgecolor = 'black', linewidth= 1)

sns.swarmplot(x='diff', y='effect', data=err_main,
              ax=ax, s=3, color = palette['grey'])

ax.axvline(x=0, color=palette['grey'], linestyle='dashed', linewidth=1)

ax.tick_params(axis='x', labelsize=12)
ax.set_xticks([0,4,8], [0,4,8])
ax.set_xlabel('Difference (deg)', fontweight='bold', fontsize=14)

ax.set_ylabel('')
ax.set_yticklabels(['Order', 'Moment'], fontsize=14,rotation=90,va='center')

ax.text(3, -0.1, 'Same vs reversed', color = palette['yellow'], fontsize = 12)
ax.text(5, 0.9, 'First vs second', color=palette['green'], fontsize = 12)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
import scipy.stats

stats = {}
stats['DT_order1'] = scipy.stats.ttest_rel(DT_all.iloc[:, 0], DT_all.iloc[:, 1])
stats['DT_order2'] = scipy.stats.ttest_rel(DT_all.iloc[:, 2], DT_all.iloc[:, 3])
stats['DT_moment'] = scipy.stats.ttest_rel(DT_all.DT1.mean(axis=1) , DT_all.DT2.mean(axis=1))

stats['err_order1'] = scipy.stats.ttest_rel(err_all.iloc[:, 0], err_all.iloc[:, 1])
stats['err_order2'] = scipy.stats.ttest_rel(err_all.iloc[:, 2], err_all.iloc[:, 3])
stats['err_moment'] = scipy.stats.ttest_rel(err_all.diff1.mean(axis=1) , err_all.diff2.mean(axis=1))

stats